# Interfacing Proxy Statements – Case Study

This notebook walks through a Retrieval-Augmented Generation (RAG) pipeline using proxy statements from Apple, Amazon, and Microsoft to extract executive compensation data and governance details. It accompanies the *Automation Ahead* article on RAG published on the Research and Policy Center (RPC) website. You can view the article [here](placeholder). You can also run the notebook in Google Colab here: [![Open In Colab](https://colab.research.google.com/drive/1_lj8tDJp9mq_gV0gzSvuMBaPoCJ3pDrm?usp=sharing).

### Key Takeaways

By the end of this notebook, you will be familiar with:

- **Building a RAG Workflow**  
  Learn how to construct a basic Retrieval-Augmented Generation pipeline for extracting structured data from complex financial documents.

- **Using Computer Vision for Metadata Extraction**  
  See how tools like Mistral’s OCR API can extract structured headers from PDFs, preserving the context needed for effective document chunking and metadata tagging.

- **Incorporating Agentic Function Calling**  
  Use function calling to integrate external tools (like a calculator) into your workflow, enabling precise outputs and reducing hallucinations for numerical tasks.

- **Evaluating Outputs with LLM-as-a-Judge**  
  Understand how to scale your evaluation process by using a large language model to judge the accuracy and relevance of other LLM outputs, especially in large datasets.

- **Analyzing Executive Compensation Details From Proxy Statements Using AI**  
  Gain practical exposure to the nuances of parsing and comparing executive compensation across companies using AI-driven methods.

# Install Dependencies
We must first install all dependencies for the notebook. You will also need a Openai and Mistral AI API key to complete this notebook. 

# Mistral AI Account Setup 
* To get started, create a Mistral account or sign in at [console.mistral.ai](https://console.mistral.ai/).
* Then, navigate to "Workspace" and "Billing" to add your payment information and activate payments on your account.
* After that, go to the "API keys" page and make a new API key by clicking "Create new key". Make sure to copy the API key, save it safely, and do not share it with anyone.

# OpenAI Account Setup
* Create an OpenAI Platform Account – Go to [platform.openai.com](https://platform.openai.com/docs/overview) and follow the prompts to sign up.
* Generate Your API Key – After signing in, navigate to the API keys page to create a new secret key.

# Securly Store Your API Keys
To securly store your api keys and access them in this notebook create a .env file in the project directory root and save the api keys as:


OPENAI_API_KEY='your Openai key'

MISTRAL_API_KEY='your Mistral AI key'

In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install pypdf
!pip install mistralai
!pip install python-dotenv
!pip install openai
!pip install tqdm
!pip install langchain_chroma
!pip install langchain_openai
!pip install pandas

## Step 1: Data Ingestion and Parsing Data
Load proxy statements from PDF files using LangChain's PyPDFLoader. These pdfs were taken from the SEC edgar website [here](https://www.sec.gov/edgar/search/#). 

In [2]:
from langchain.document_loaders import PyPDFLoader
import os

# Directory where PDFs are stored
pdf_folder = "../Retrieval Augmented Generation/DEF14A-PDFs"

# Get list of PDF files in the folder
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]

# Load each PDF into a list of documents
documents = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    documents.extend(docs)

# To parse the pdfs we will use Mistral's OCR API.
We use Mistral’s Optical Character Recognition (OCR) API because traditional parsing methods often miss key structural elements like headings. The Mistral's OCR API model has been trained to include these headings in its output which give an LLM key structural elements that can be used as metadata that enhance the LLMs ability to answer questions.  

In [3]:
#Structured Parsing with OCR
#Use Mistral’s OCR API to extract structured markdown text from PDFs.

from mistralai import Mistral
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [5]:
from pathlib import Path
import json
from mistralai import DocumentURLChunk
from tqdm import tqdm


def parse_pdf(client: object, pdf_path: Path) -> str:
    # Upload PDF to Mistral's OCR service
    uploaded_file = client.files.upload(
        file={
            "file_name": pdf_path.stem,
            "content": pdf_path.read_bytes()
        },
        purpose="ocr"
    )
    
    # Get URL for the uploaded file from Mistral API
    signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

    # Process PDF with OCR, returning markdown text
    pdf_response = client.ocr.process(
        document=DocumentURLChunk(document_url=signed_url.url),
        model="mistral-ocr-latest"
    )

    # Extract and return the markdown content
    response_dict = json.loads(pdf_response.model_dump_json())
    markdown_text = "\n\n".join(page["markdown"] for page in response_dict["pages"])
    
    return markdown_text


# Batch process multiple PDFs and save results
def batch_process_pdfs_to_markdown(client, input_dir: str, output_dir: str) -> dict:
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    markdown_results = {}

    pdf_files = list(input_path.glob("*.pdf"))

    for pdf in tqdm(pdf_files, desc="Processing PDFs"):
        try:
            md_text = parse_pdf(client, pdf)
            markdown_results[pdf.stem] = md_text

            # Save each markdown file separately
            md_file_path = output_path / f"{pdf.stem}.md"
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(md_text)

        except Exception as e:
            print(f"Failed to process {pdf.name}: {e}")

    return markdown_results

In [ ]:
input_dir = "../Retrieval Augmented Generation/DEF14A-PDFs"
output_dir = "../Retrieval Augmented Generation/markdown_outputs"

markdown_data = batch_process_pdfs_to_markdown(client, input_dir, output_dir)

## Step 2: Chunking the Markdown Files
Here we use the structured headers created by the Mistral model to chunk documents. This allows us to keep the same semantic structure of the original document. 

In [7]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

def split_markdown_by_headers(pdf_name: str, markdown: str) -> list:
    # Define header levels to split on
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3")
    ]
    
    # Initialize the Markdown splitter
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
    
    # Split the markdown content based on headers
    md_header_splits = markdown_splitter.split_text(markdown)
    
    # Attach the original file name as metadata for traceability
    for split in md_header_splits:
        split.metadata['file_name'] = pdf_name
    
    return md_header_splits

In [8]:
def batch_split_markdown_files(input_dir: str) -> list:
    input_path = Path(input_dir)
    md_files = list(input_path.glob("*.md"))

    all_chunks = []

    for md_file in tqdm(md_files, desc="Splitting Markdown Files"):
        with open(md_file, "r", encoding="utf-8") as f:
            markdown_content = f.read()
        
        chunks = split_markdown_by_headers(md_file.stem, markdown_content)
        all_chunks.extend(chunks)
    
    return all_chunks

In [ ]:
markdown_input_dir = "../Retrieval Augmented Generation/markdown_outputs"
md_header_splits = batch_split_markdown_files(markdown_input_dir)


## Step 3: Embedding and Storing Documents in Vector Database
We now embed the chunked documents and store them in a vector database for LLM retrieval.

In [11]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Store document embeddings in a vector database
vectorstore = Chroma.from_documents(
    documents=md_header_splits,
    embedding=OpenAIEmbeddings(),
    persist_directory="vector_db_md_split"
)

In [ ]:
# You can also load the vectorstore from a directory if already done the previous step. Unhash the below if so.

# from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

# vectorstore = Chroma(persist_directory="vector_db_md_split", embedding_function=OpenAIEmbeddings())

## Step 4: Define the Retriever and Setup the Prompt


In [12]:
from langchain_core.prompts import ChatPromptTemplate

# Define your prompt template explicitly
template = """Execute the following query based only on the following context.

Query: \n {query}

Context: \n {context}

"""

# Create a retriever from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 100})

def create_rag_prompt(query: str, retriever, template: str) -> str:

		# Create the prompt object from the prompt template
    prompt = ChatPromptTemplate.from_template(template)
    
    # Retrieve relevant documents (context) for a given query
    context_docs = retriever.get_relevant_documents(query)
    
    # Format context documents into a single string
    context_str = str(context_docs)
    
    # Format the final prompt we will send to the Openai chat completions API with context and query
    final_prompt = prompt.format(context=context_str, query=query)

    return final_prompt

## Step 5: Structured Response Generation using OpenAI
Here we give the LLM a (pydantic) structured model called ExplainOutputs so that we can easily parse the answer. Doing so provides more control in our RAG system and reduces the likelihood of the model to provide inconsistent structure in its answer. We also ask the LLM to explain its answer before arriving at it, providing more reasoning power to arrive at the correct answer. 

In [13]:
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel

# Define the structured output
class ExplainOutputs(BaseModel):
	explanation: str
	answer: str

# Define the OpenAI API client and load environment API keys
client = OpenAI()
load_dotenv()

def run_rag_query(query:str,retriever,structured_response_model,model):
    # Create the prompt using the provided query
    final_prompt = create_rag_prompt(
        query=query,
        retriever=retriever,
        template=template
    )
    # Send the structured prompt to OpenAI's ChatCompletion API
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert at interfacing documents and providing responses in a structured format."},
            {"role": "user", "content": final_prompt}
        ],
        response_format=structured_response_model,  # Enforces structured output
    )
    structured_response = completion.choices[0].message.parsed  
    return structured_response

# Testing Single Query Responses
Here we test the LLMs ability to answer single queries on the documents.

In [17]:
questions = [
    "Does Apple have a dedicated committee in place to oversee artificial intelligence?",
    "What is Microsoft’s policy on executives’ derivatives trading?",
    "List executive board members and independent board members for Amazon to be elected for the next fiscal period",
    "What was Satya Nadella’s Total Compensation without the inclusion of other compensation? Give it to me as % of Total Compensation to the 4th decimal place."
]


In [19]:
from typing import List

# We batch run the queries to the RAG system for demonstration purposes
def run_batch_rag_queries(questions: List[str], retriever,structured_response_model,model="gpt-4o-mini"):
    results = []
    for q in questions:
        result = run_rag_query(q, retriever, structured_response_model,model=model)
        results.append(result)
    return results

responses = run_batch_rag_queries(questions, retriever,ExplainOutputs, model="gpt-4.1-mini")


/var/folders/3g/79hl_7s12859plnmqc8l6nrw0000gp/T/ipykernel_97244/470427115.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(query)


In [ ]:
for query,response in zip(questions,responses):
    print(f"Question: {query}")
    print(f"Answer: {response.answer}")
    print('\n')

# Testing Function Calling and Agent Integration For RAG Workflow
While the last answer was close to the correct one, which was 99.7854%, we can see that the LLM hallucinated the exact digits of the actual answer. This happens because LLMs rely on prior knowledge to answer questions rather than performing fundamental arithmetic. 

Now we test to see if by giving the LLM access to a "calculator" (in this case a python function) if it can answer correctly. This is an example of using function calling agents which provides the LLM a function to excecute when it needs to perform arithmetic. 

In [ ]:
pip install openai-agents # or `uv add openai-agents`, etc

In [36]:
from agents import Agent,function_tool, Runner
from pydantic import BaseModel, Field

# The next two classes define the data structure of the input and output for the calculator tool
class MathExpression(BaseModel):
    expression: str = Field(..., description="A valid Python mathematical expression, e.g., '(2.5 + 3) * 4 / (1.5 ** 2)'")

class MathExpressionResponse(BaseModel):
    result: str = Field(..., description="The result of the evaluated expression as a string.")

@function_tool
async def calculator_tool(data: MathExpression) -> str:
    """
    Evaluates a mathematical expression using Python's eval() and returns the result as a string.

    Args:
        data (MathExpression): An object containing a math expression to evaluate (e.g., '(2 + 3) * 4').

    Returns:
        str: The result of the evaluated expression, formatted as a string.
    """
    try:
        # Optional: restrict available built-ins for safety
        result = eval(data.expression, {"__builtins__": None}, {})
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"


In [ ]:
# Here we define the agents that will use the calculator tool and the expression agent
expression_agent = Agent(
    name="Math Expression Agent",
    instructions="You are a math assistant specialized in creating mathematical expressions that work with python's eval function.",
    output_type=MathExpression,
    model='gpt-4.1-mini'
)

expression_agent_tool = expression_agent.as_tool(
    tool_name="expression_agent_tool",
    tool_description="Create a mathematical expression that will work with pythons eval function from the input."
)

calculator_agent = Agent(
    name="Financial Math Agent",
    instructions="You are a math assistant specialized in financial analysis. Ceate a mathematical expression and use the calculator_tool for the precise calculations.",
    output_type=MathExpressionResponse,
    tools=[calculator_tool],
    model='gpt-4.1-mini'
)

calculator_agent_tool = calculator_agent.as_tool(
    tool_name="calculator_agent_tool",
    tool_description="Evaluate mathematical expressions using Python for financial questions."
)

triage_agent = Agent(
    name="triage_agent",
    instructions="You are a triage agent specialized in financial analysis. You will triage the question to determine which agent to pass the request off to.",
    tools=[calculator_agent_tool, expression_agent_tool],
    model='gpt-4.1-mini'
)


In [38]:
query = "Agent, what is the total compensation of Satya Nadella without the inclusion of other compensation? Give it to me as % of Total Compensation to the 4th decimal place."
prompt = create_rag_prompt(
        query=query,
        retriever=retriever,
        template=template
    )



In [ ]:
result = await Runner.run(triage_agent, prompt)
print(result.final_output)

# Testing Realized Compensation

In [ ]:
question = 'What is Tim Cook’s realized long-term Incentives for the fiscal year 2024 based on shares vested?'
response = run_rag_query(question, retriever, structured_response_model=ExplainOutputs, model="gpt-4.1-mini",)
print(f"Question: {question}")
print(f"Answer: {response.answer}")

# Automated Multi-Company Multi-Variable Extraction
We now test the limitations of the RAG workflows by giving it a much more complex task which included extracting multiple variables from multiple companies and providing the answer in a structured table for analysis.  

In [41]:
companies = ['Apple', 'Amazon', 'Microsoft']

# Create the prompt

In [42]:
query = f"""You are an expert financial analysis assistant specializing in extracting detailed executive compensation data from proxy statements. Your task is to extract the following fields from the provided proxy statements of {",".join(companies)} and output the information in the given structure. For any missing or not applicable information, use “N/A”.

Extract these fields for each company:
1. Company Name: Full legal name of the company.
2. Company CEO: Name of the Chief Executive Officer.
3. Coverage Period: The fiscal period covered by the proxy (e.g., "Fiscal Year 2024" or "Fiscal Year Ended June 30, 2024").
4. Total Target LTI (Full Grant Amount): The total target long-term incentive amount, including all equity components.
5. LTI Package Grant Date: The date when the LTI grant was awarded.
6. Annual LTI Grant?: Indicate “Yes” if equity awards are granted annually, or “No” if not.
7. Time-Based RSU Vesting Schedule: Details on vesting for time-based equity (e.g., "Three equal annual installments starting April 1, 2026" or quarterly vesting over a specified period). If not applicable, indicate “N/A”.
8. Performance-Based RSU Vesting Schedule: Details on vesting for performance-based equity, including the performance period and conditions (e.g., "Vests on October 1, 2026 based on performance from October 1, 2023 to September 30, 2026"). If not applicable, indicate “N/A”.
9. Compensation Governance Arrangements: Information on the oversight mechanisms (e.g., independent compensation committees, clawback provisions, and stock ownership policies).
10. CEO Pay Alignment Mechanisms: How the compensation is structured to align CEO pay with long-term shareholder value.
11. Performance Metrics Used (Detailed): Provide specifics of the performance metrics applied, including the measurement period and targets (e.g., "Relative TSR compared to the S&P 500 over a 3-year period (October 2023 – October 2026), payout ranges from 0%–200%" or details on revenue growth targets).
12. Realized Base Salary: Actual base salary paid in the period.
13. Realized STIs: Actual short-term incentives (bonuses) paid.
14. Realized Long-Term Awards: Value of equity awards that have vested in the period.
15. Realized Other Compensation: Additional benefits (e.g., security costs, deferred compensation, change-in-control benefits).
16. Realized Total Compensation: The sum of all compensation elements actually received in the period.

Output your findings in a JSON format.
"""

# Create the Stuctured Output Model
Here we create the structured model for the output to be converted into a pandas DataFrame.

In [43]:
from pydantic import BaseModel, Field
from typing import Optional, List
from datetime import date

class ExecutiveCompensation(BaseModel):
    company_name: str
    company_ceo: str
    coverage_period: str
    total_target_lti: Optional[float] = Field(..., description="Target LTI in USD")
    lti_grant_date: Optional[str]
    annual_lti_grant: Optional[bool]
    time_based_rsu_vesting_schedule: Optional[str]
    performance_based_rsu_vesting_schedule: Optional[str]
    compensation_governance_arrangements: Optional[str]
    ceo_pay_alignment_mechanisms: Optional[str]
    performance_metrics_used: Optional[str]
    realized_base_salary: Optional[float]
    realized_stis: Optional[float]
    realized_long_term_awards: Optional[float]
    realized_other_compensation: Optional[float]
    realized_total_compensation: Optional[float]

class ExecutiveCompensationReport(BaseModel):
    companies: List[ExecutiveCompensation]

In [44]:
executive_compensation_report = run_rag_query(query, retriever, structured_response_model=ExecutiveCompensationReport, model="gpt-4.1-mini")

In [45]:
import pandas as pd

executive_compensation_report_json = executive_compensation_report.model_dump()
executive_compensation_report_db = pd.DataFrame(executive_compensation_report_json['companies'])

In [ ]:
executive_compensation_report_db

# Evaluating LLM Responses Using an LLM Judge
We now evaluate the response using an LLM as a Judge. Here we compare the groundtruth labels which were completed manually by a human, against the LLM outputed answers.


In [82]:
import pandas as pd
ground_truth_db = pd.read_excel('../Retrieval Augmented Generation/ground_truth_table.xlsx')

In [83]:
# Fist, we need to convert both dataframes from wide to long format so we can merge both dataframes and compare them horizontally
ground_truth_db_melted = ground_truth_db.melt(id_vars=['company_name'], var_name='field', value_name='ground_truth_db_value')
executive_compensation_report_melted = executive_compensation_report_db.melt(id_vars=['company_name'], var_name='field', value_name='exectuive_compensation_report_value')

In [84]:
# Now we can merge the two dataframes on company_name and field
combined_db = ground_truth_db_melted.merge(executive_compensation_report_melted, on=['company_name', 'field'])

# Create the Judge
Here we create the output model and the LLM judge used for evaluating the answers against the ground truth labels.

In [52]:
# Define the updated structured output model with explanation
class EvaluationOutput(BaseModel):
    field: str
    model_answer: str
    ground_truth: str
    score: float
    explanation: str

# Define the function to run the LLM Judge query
def run_llm_judge_query(query: str, field:str, model: str, ground_truth: dict, model_generated: dict, structured_response_model: EvaluationOutput):
    # Create the prompt for the LLM Judge to compare answers
    prompt = f"""
    Evaluate the accuracy of the model's response for the following fields by comparing it to the ground truth. 
    For each field, provide a score between 0 and 1:
    0 means completely incorrect or missing.
    1 means fully correct and accurate.
    
    Additionally, provide an explanation of why the score was given.
    
    Query: {query}

    Field: {field}
    
    Model Answer: {model_generated}
    Ground Truth: {ground_truth}
    
    Compare the two and assign a score for each field, along with an explanation for the score.
    Return the score and explanation in structured JSON format.
    """

    # Send the structured prompt to OpenAI's ChatCompletion API
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert evaluator comparing model outputs to ground truth."},
            {"role": "user", "content": prompt}
        ],
        response_format=structured_response_model,  # Enforces structured output
    )
    structured_response = completion.choices[0].message.parsed

    return structured_response


In [53]:
evaluations_results = []

for index, row in combined_db.iterrows():
    field = row['field']
    model_answer = row['exectuive_compensation_report_value']
    ground_truth = row['ground_truth_db_value']

    # Run the LLM Judge query
    evaluation_result = run_llm_judge_query(
        query=query,
        field=field,
        model="gpt-4.1-mini",
        ground_truth=ground_truth,
        model_generated=model_answer,
        structured_response_model=EvaluationOutput
    )
    evaluations_results.append(evaluation_result)


In [85]:
# Here we create a new column for the LLM score on our combined dataframe
combined_db['executive_compensation_score'] = [evaluations_result.score for evaluations_result in evaluations_results]

In [86]:
# Now we can see the results horizontally
combined_db


,company_name,field,ground_truth_db_value,exectuive_compensation_report_value,executive_compensation_score
0,"Amazon.com, Inc.",company_ceo,Andrew R. Jassy,Andrew R. Jassy,1.00
1,Apple Inc.,company_ceo,Tim Cook,Tim Cook,1.00
2,Microsoft Corporation,company_ceo,Satya Nadella,Satya Nadella,1.00
3,"Amazon.com, Inc.",coverage_period,Fiscal Year 2023,"Fiscal Year Ended December 31, 2023",0.90
4,Apple Inc.,coverage_period,Fiscal Year 2024,Fiscal Year 2024,1.00
5,Microsoft Corporation,coverage_period,Fiscal Year 2024,"Fiscal Year Ended June 30, 2024",0.90
6,"Amazon.com, Inc.",total_target_lti,211933520,NaN,0.00
7,Apple Inc.,total_target_lti,50000000,50000000.0,1.00
8,Microsoft Corporation,total_target_lti,50000000,50000000.0,1.00
9,"Amazon.com, Inc.",lti_grant_date,2021,N/A,0.00


# Plot the Scores as a Heatmap
Lastly, we can visualize our LLM accuracy using a heatmap for each variable and company

In [ ]:
!pip install matplotlib
!pip install seaborn


In [92]:
combined_db

,company_name,field,ground_truth_db_value,exectuive_compensation_report_value,executive_compensation_score
0,"Amazon.com, Inc.",company_ceo,Andrew R. Jassy,Andrew R. Jassy,1.00
1,Apple Inc.,company_ceo,Tim Cook,Tim Cook,1.00
2,Microsoft Corporation,company_ceo,Satya Nadella,Satya Nadella,1.00
3,"Amazon.com, Inc.",coverage_period,Fiscal Year 2023,"Fiscal Year Ended December 31, 2023",0.90
4,Apple Inc.,coverage_period,Fiscal Year 2024,Fiscal Year 2024,1.00
5,Microsoft Corporation,coverage_period,Fiscal Year 2024,"Fiscal Year Ended June 30, 2024",0.90
6,"Amazon.com, Inc.",total_target_lti,211933520,NaN,0.00
7,Apple Inc.,total_target_lti,50000000,50000000.0,1.00
8,Microsoft Corporation,total_target_lti,50000000,50000000.0,1.00
9,"Amazon.com, Inc.",lti_grant_date,2021,N/A,0.00


In [101]:
import matplotlib.pyplot as plt
import seaborn as sns

def create_accuracy_heatmap(df: pd.DataFrame):
    df = df.copy()

    # Dictionary to rename fields for better readability
    field_rename_map = {
        "company_ceo": "CEO",
        "coverage_period": "Coverage Period",
        "total_target_lti": "Total Target LTI",
        "lti_grant_date": "LTI Grant Date",
        "annual_lti_grant": "Annual LTI Grant?",
        "time_based_rsu_vesting_schedule": "Time-Based RSU Vesting",
        "performance_based_rsu_vesting_schedule": "Performance-Based RSU Vesting",
        "compensation_governance_arrangements": "Governance Arrangements",
        "ceo_pay_alignment_mechanisms": "CEO Pay Alignment",
        "performance_metrics_used": "Performance Metrics",
        "realized_base_salary": "Realized Base Salary",
        "realized_stis": "Realized STIs",
        "realized_long_term_awards": "Realized LT Awards",
        "realized_other_compensation": "Realized Other Compensation",
        "realized_total_compensation": "Realized Total Compensation"
    }

    # Rename the 'field' column according to the mapping
    df['field'] = df['field'].map(field_rename_map)

    # Define the correct order of fields
    correct_field_order = [
        "CEO", "Coverage Period", "Total Target LTI", "LTI Grant Date", "Annual LTI Grant?",
        "Time-Based RSU Vesting", "Performance-Based RSU Vesting", "Governance Arrangements",
        "CEO Pay Alignment", "Performance Metrics", "Realized Base Salary", "Realized STIs",
        "Realized LT Awards", "Realized Other Compensation", "Realized Total Compensation"
    ]

    # Ensure the 'field' column has the correct order
    df['field'] = pd.Categorical(df['field'], categories=correct_field_order, ordered=True)
    
    # Pivot the DataFrame to get company names as rows and fields as columns
    pivot_df = df.pivot(index="company_name", columns="field", values="executive_compensation_score")
    
    # Create the heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_df, annot=True, cmap="RdYlGn", linewidths=0.5, cbar_kws={'label': 'Accuracy Score'}, vmin=0, vmax=1)
    plt.title("LLM Accuracy Heatmap for Executive Compensation Extraction")
    plt.xlabel("Variable")
    plt.ylabel("Company Name")
    plt.show()

In [ ]:
create_accuracy_heatmap(combined_db)

### Note scores will depend on the models used. The original article on the Research and Policy Center website used gpt4o-mini. Your results may differ. 